Convolution and Pooling Layers
 
Input Layer -> 1st Convolutional Layer -> 1st Pooling Layer -> 2nd Convolutional Layer -> 2nd Pooling Layer -> 3rd Convolutional Layer -> 3rd Pooling Layer -> Fully Connected Layer -> Output Layer


In [ ]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

# Add function to create weight variable 
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

# Add function to create bias variable 
def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

# Add convolution function
def conv2d(x, W):
    return(tf.nn.conv2d(x, W, strides = [1, 1, 1, 1], padding = "SAME"))

# Add Pooling function
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")

# Reshape the data for inputting it in the input layer
x_image = tf.reshape(x, [-1, 28, 28, 1])

# Create 1st Convolutional and Pooling Layer
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)


# Create 2nd Convolutional and Pooling Layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)


# Create 3rd Convolutional and Pooling Layer
W_conv3 = weight_variable([5, 5, 64, 128])
b_conv3 = bias_variable([128])

h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)

# 1st Fully Connected Layer
W_fc1 = weight_variable([128*4*4, 1024])
b_fc1 = bias_variable([1024])

# Reshape the last convolutional layer output to connect with Fully Connected Layer
h_pool2_flat = tf.reshape(h_pool3, [-1, 128*4*4])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Add the dropout layer
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Output Layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


In [ ]:
# Training 

In [ ]:
# Network base of Computation
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = y_conv))
train_step  = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Prediction Measures
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Mention about the number of epochs training has to happen
epochs = 1100

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epochs):
        # Read a batch of training Data
        batch =  mnist.train.next_batch(50)
        if(i%100 == 0):
            train_accuracy = accuracy.eval(feed_dict = {x: batch[0], y_:batch[1], keep_prob : 1.0})
            print('step %d, training accuracy %g' %(i, train_accuracy))
        train_step.run(feed_dict = {x: batch[0], y_:batch[1], keep_prob :0.5})    
    for i in range(epochs):
        # Read a batch of testing Data
        batch =  mnist.test.next_batch(50)
        if(i%100 == 0):
            test_accuracy = accuracy.eval(feed_dict = {x: batch[0], y_:batch[1], keep_prob : 1.0})
            print('step %d, testing accuracy %g' %(i, test_accuracy))
